In [1]:
import keras

In [2]:
dataset_path = 'dataset'

train_path = dataset_path + '/train'
train_images_path = train_path + '/images'
train_annotations_path = train_path + '/annotations'

valid_path = dataset_path + '/valid'
valid_images_path = valid_path + '/images'
valid_annotations_path = valid_path + '/annotations'

test_path = dataset_path + '/test'
test_images_path = test_path + '/images'
test_annotations_path = test_path + '/annotations'

In [ ]:
# images are .png images
# annotations are .json files
# and have the following format:
"""
{
    "version": "5.4.1",
    "flags": {},
    "shapes": [
        {
            "label": "femurR",
            "points": [
                [
                    598.125,
                    892.5
                ]
            ],
            "group_id": null,
            "description": "",
            "shape_type": "point",
            "flags": {},
            "mask": null
        },
        {
            "label": "femurR",
            "points": [
                [
                    658.125,
                    903.125
                ]
            ],
            "group_id": null,
            "description": "",
            "shape_type": "point",
            "flags": {},
            "mask": null
        },
        {
            "label": "femurR",
            "points": [
                [
                    670.625,
                    983.75
                ]
            ],
            "group_id": null,
            "description": "",
            "shape_type": "point",
            "flags": {},
            "mask": null
        },
        {
            "label": "femurL",
            "points": [
                [
                    1170.0,
                    892.5
                ]
            ],
            "group_id": null,
            "description": null,
            "shape_type": "point",
            "flags": {},
            "mask": null
        },
        {
            "label": "femurL",
            "points": [
                [
                    1115.0,
                    894.375
                ]
            ],
            "group_id": null,
            "description": "",
            "shape_type": "point",
            "flags": {},
            "mask": null
        },
        {
            "label": "femurL",
            "points": [
                [
                    1086.25,
                    969.375
                ]
            ],
            "group_id": null,
            "description": null,
            "shape_type": "point",
            "flags": {},
            "mask": null
        },
        {
            "label": "acetabulumR",
            "points": [
                [
                    611.25,
                    871.25
                ]
            ],
            "group_id": null,
            "description": "",
            "shape_type": "point",
            "flags": {},
            "mask": null
        },
        {
            "label": "acetabulumL",
            "points": [
                [
                    1148.75,
                    858.75
                ]
            ],
            "group_id": null,
            "description": "",
            "shape_type": "point",
            "flags": {},
            "mask": null
        }
    ],
    "imageHeight": 2140,
    "imageWidth": 1760
}
"""
# the model should be able to receive the image and predict the points for all the labels

In [3]:
# here we put the valid images in the train directory
# because we are doing the split during the training with validation_split = 0.2

import shutil
import os

# Define the source and destination directories
valid_images_path = os.path.join(dataset_path, 'valid', 'images')
train_images_path = os.path.join(dataset_path, 'train', 'images')

# Get a list of all the image files in the valid directory
valid_image_files = os.listdir(valid_images_path)

# Move each file to the train directory
for file_name in valid_image_files:
    shutil.move(os.path.join(valid_images_path, file_name), train_images_path)

In [4]:
import os
import json
import cv2
import numpy as np
from glob import glob
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Define the splits
splits = ['train', 'valid', 'test']

# Function to sort points
def sort_points(annotation):
    # Define the order of labels
    label_order = ['femurR', 'femurL', 'acetabulumR', 'acetabulumL']

    # Create a list of tuples, each containing a point and its corresponding label
    points_with_labels = [(point, shape['label']) for shape in annotation['shapes'] for point in shape['points']]
    # Sort the points based on the predefined label order
    points_with_labels.sort(key=lambda x: label_order.index(x[1]))
    # Remove the labels, leaving only the sorted points
    sorted_points = [point for point, label in points_with_labels]

    return sorted_points

# Function to load images and annotations
def load_data(images_path, annotations_path):
    # Get a list of all image files and annotation files
    image_files = sorted(glob(os.path.join(images_path, '*.png')))
    annotation_files = sorted(glob(os.path.join(annotations_path, '*.json')))

    #print(f'Found {len(image_files)} image files and {len(annotation_files)} annotation files.')

    images = []
    annotations = []
    image_paths = []

    for image_file, annotation_file in zip(image_files, annotation_files):
        # Load and resize the image
        image = load_img(image_file, target_size=(64, 64))
        image = img_to_array(image)
        images.append(image)

        # Load and process the annotation
        with open(annotation_file, 'r') as f:
            annotation = json.load(f)
        points = sort_points(annotation)
        annotations.append(points)

        # Save the image path
        image_paths.append(image_file)

    return np.array(images), np.array(annotations), image_paths

# Load data for each split
for split in splits:
    images_path = os.path.join(dataset_path, split, 'images')
    annotations_path = os.path.join(dataset_path, split, 'annotations')
    split_images, split_annotations, split_image_paths = load_data(images_path, annotations_path)

    if split == 'train':
        train_images, train_annotations, train_image_paths = split_images, split_annotations, split_image_paths
    elif split == 'valid':
        valid_images, valid_annotations, valid_image_paths = split_images, split_annotations, split_image_paths
    elif split == 'test':
        test_images, test_annotations, test_image_paths = split_images, split_annotations, split_image_paths

    print(f'Loaded {len(split_images)} images and {len(split_annotations)} annotations for {split}.')

Loaded 115 images and 115 annotations for train.
Loaded 0 images and 0 annotations for valid.
Loaded 17 images and 17 annotations for test.


In [8]:
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# Load the pre-trained ResNet50V2 model, excluding the top layer
base_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully-connected layer
x = Dense(1024, activation='relu')(x)

# Determine the number of points in the annotations
num_points = train_annotations.shape[1]

# Load the pre-trained ResNet50V2 model, excluding the top layer
base_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully-connected layer
x = Dense(1024, activation='relu')(x)

# Add the output layer
predictions = Dense(num_points)(x)

# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

# Define the optimizer with a specific learning rate
optimizer = Adam(learning_rate=0.001)

# Compile the model
model.compile(optimizer=optimizer, loss='mse')

# Print the model summary
model.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 64, 64, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 70, 70, 3) │          0 │ input_layer_3[0]… │
│ (ZeroPadding2D)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 32, 32,    │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 34, 34,    │          0 │ conv1_conv[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 16, 16,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_preac… │ (None, 16, 16,    │        256 │ pool1_pool[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_preac… │ (None, 16, 16,    │          0 │ conv2_block1_pre… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 16, 16,    │      4,096 │ conv2_block1_pre… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 16, 16,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 16, 16,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_pad  │ (None, 18, 18,    │          0 │ conv2_block1_1_r… │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 16, 16,    │     36,864 │ conv2_block1_2_p… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 16, 16,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 16, 16,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 16, 16,    │     16,640 │ conv2_block1_pre… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 16, 16,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_out    │ (None, 16, 16,    │          0 │ conv2_block1_0_c

 Total params: 25,679,376 (97.96 MB)

 Trainable params: 25,633,936 (97.79 MB)

 Non-trainable params: 45,440 (177.50 KB)

In [6]:
# Just to check how the model is receving all the data

# Convert the lists to numpy arrays
train_images = np.array(train_images)
train_annotations = np.array(train_annotations)

# Print the first image and annotation before processing
print("Before processing:")
print("First image path:", train_image_paths[0])
print("First image shape:", train_images[0].shape)
print("First image data:", train_images[0])
print("First annotation shape:", train_annotations[0].shape)
print("First annotation data:", train_annotations[0])

# Reshape the annotations and normalize the images
train_annotations = train_annotations.reshape(len(train_annotations), -1)
train_images = train_images / 255.0

# Print the first image and annotation after processing
print("\nAfter processing:")
print("First image path:", train_image_paths[0])
print("First image shape:", train_images[0].shape)
print("First image data:", train_images[0])
print("First annotation shape:", train_annotations[0].shape)
print("First annotation data:", train_annotations[0])

Before processing:
First image path: dataset/train/images/0001_1_PF.png
First image shape: (64, 64, 3)
First image data: [[[ 5.  5.  5.]
  [ 4.  4.  4.]
  [ 3.  3.  3.]
  ...
  [ 3.  3.  3.]
  [ 3.  3.  3.]
  [ 3.  3.  3.]]

 [[ 4.  4.  4.]
  [ 4.  4.  4.]
  [ 3.  3.  3.]
  ...
  [ 3.  3.  3.]
  [ 3.  3.  3.]
  [ 4.  4.  4.]]

 [[ 4.  4.  4.]
  [ 4.  4.  4.]
  [ 3.  3.  3.]
  ...
  [ 3.  3.  3.]
  [ 3.  3.  3.]
  [ 4.  4.  4.]]

 ...

 [[ 3.  3.  3.]
  [ 2.  2.  2.]
  [ 0.  0.  0.]
  ...
  [ 2.  2.  2.]
  [ 2.  2.  2.]
  [ 3.  3.  3.]]

 [[ 2.  2.  2.]
  [ 2.  2.  2.]
  [ 0.  0.  0.]
  ...
  [ 0.  0.  0.]
  [ 2.  2.  2.]
  [ 3.  3.  3.]]

 [[46. 46. 46.]
  [41. 41. 41.]
  [36. 36. 36.]
  ...
  [ 2.  2.  2.]
  [ 2.  2.  2.]
  [ 3.  3.  3.]]]
First annotation shape: (8, 2)
First annotation data: [[ 598.125  892.5  ]
 [ 658.125  903.125]
 [ 670.625  983.75 ]
 [1170.     892.5  ]
 [1115.     894.375]
 [1086.25   969.375]
 [ 611.25   871.25 ]
 [1148.75   858.75 ]]

After processing:
First i

In [9]:
# Convert the lists to numpy arrays
train_images = np.array(train_images)
train_annotations = np.array(train_annotations).reshape(len(train_annotations), -1)

# Normalize the images
train_images = train_images / 255.0

# Train the model
print(f'Training on {len(train_images)} images')
model.fit(train_images, train_annotations, epochs=30, validation_split=0.2, batch_size=32)

Training on 115 images
Epoch 1/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 11s 742ms/step - loss: 1562954.0000 - val_loss: 1128908.2500
Epoch 2/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 422ms/step - loss: 1491212.1250 - val_loss: 1040668.1875
Epoch 3/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 432ms/step - loss: 1458049.5000 - val_loss: 2527352.2500
Epoch 4/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 451ms/step - loss: 1464766.6250 - val_loss: 19985326.0000
Epoch 5/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 473ms/step - loss: 1277643.0000 - val_loss: 3585773.2500
Epoch 6/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 437ms/step - loss: 1243548.5000 - val_loss: 159159.9219
Epoch 7/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 436ms/step - loss: 1122398.7500 - val_loss: 770897.7500
Epoch 8/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 434ms/step - loss: 1109613.8750 - val_loss: 1018519.1875
Epoch 9/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 460ms/step - loss: 1018348.0000 - val_loss: 1032809.9375
Epoch 10/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 448ms/step - loss: 893432.7500 - val_loss: 979350.2500
Epoch 11/30
3/3 ━━